In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

%cd "/content/gdrive/My Drive/DeepfakeTest/first-order-model"

from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte

path_pic = '../Data/face_AI.png'
path_mov = '../Data/face_ME.mp4'

source_image = imageio.imread(path_pic)
reader = imageio.get_reader(path_mov)
driving_video = imageio.mimread(path_mov)

#Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', checkpoint_path='../Data/vox-cpk.pth.tar', cpu=True)
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True, cpu=True)
imageio.mimsave('../Data/generated1.mp4', [img_as_ubyte(frame) for frame in predictions], fps=reader.get_meta_data()['fps'])

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True, cpu=True)
imageio.mimsave('../Data/generated2.mp4', [img_as_ubyte(frame) for frame in predictions], fps=reader.get_meta_data()['fps'])